In [1]:
import numpy as np                           #importing required libraries
import pandas as pd
import re
from nltk.corpus import stopwords
from random import randrange
from itertools import islice


In [2]:
#loading the data

data=pd.read_csv('data/amazon_cells_labelled.txt' , delimiter='\t', names=['review', 'sentiment'])

In [3]:
data.head()


,review,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     1000 non-null   object
 1   sentiment  1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [5]:
data['sentiment'].value_counts()

0    500
1    500
Name: sentiment, dtype: int64

In [6]:
def text_filtering(text):
    #stop_words = list(stopwords.words('english'))
    stop_words=[]
    #stop_words=['those','is','and','a','at','these','for','these']
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    for word in text.split(" "):
        if word in stop_words:
            text=text.replace(word,'')
    text = text.rstrip() 
    regular_expression = '[^a-zA-z\s]'
    text = re.sub(regular_expression, '', text)
    return text
text_filtering("")

''

In [7]:
train_data = data[:int(0.9*len(data))]
test_and_dev_data = data[int(0.9*len(data)):]
                                                                       #splitting the data

dev_data=test_and_dev_data[:int(0.5 * len(test_and_dev_data))]
test_data=test_and_dev_data[int(0.5 * len(test_and_dev_data)):]

print(train_data.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     900 non-null    object
 1   sentiment  900 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.2+ KB
None


In [8]:
print(test_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 950 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50 non-null     object
 1   sentiment  50 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 932.0+ bytes
None


In [9]:
print(dev_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 900 to 949
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50 non-null     object
 1   sentiment  50 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 932.0+ bytes
None


In [10]:
train_data['sentiment'].value_counts()

1    458
0    442
Name: sentiment, dtype: int64

In [11]:
test_data['sentiment'].value_counts()

0    33
1    17
Name: sentiment, dtype: int64

In [12]:
dev_data['sentiment'].value_counts()

0    25
1    25
Name: sentiment, dtype: int64

In [13]:
X_train = train_data['review']
y_train = train_data['sentiment']
# print(X_train)
# print(y_train)

X_dev = dev_data['review']
y_dev = dev_data['sentiment']
# print(X_dev)
# print(y_dev)

X_test = test_data['review']
y_test = test_data['sentiment']
# print(X_test)
# print(y_test)

In [14]:

def cal_prior_probs(y_train):
    prior_probs = []
    count_0 = sum(y_train == 0)
    prior_probs.append(count_0 / y_train.size)  
    count_1 = sum(y_train == 1)
    prior_probs.append(count_1 / y_train.size)    
    return prior_probs
prior_probs=cal_prior_probs(y_train)
print("Negative review class' prior probability--",prior_probs[0] )
print("Positive review class' prior probability--",prior_probs[1] )

Negative review class' prior probability-- 0.4911111111111111
Positive review class' prior probability-- 0.5088888888888888


In [15]:
def build_word_dict(data):
    word_freq = {}
    all_words=[]
    omit=[]
    for sentence in data:
            words=[]
            sentence = text_filtering(sentence)
            words=sentence.split(' ')
            for word in set(words) :
                word = word.lower()
                if word not in word_freq.keys():
                    word_freq[word] = 1
                    all_words.append(word)
                else:
                    word_freq[word] += 1
#     for word in word_freq:
#         if word_freq[word] <5:
#             omit.append(word)

#     for word in omit:
#         word_freq.pop(word,None)
        
            

    return word_freq, all_words

    
       
allwords_freq,allwords_train=build_word_dict(X_train)
print(allwords_train)
allwords_freq


# train_pos_reviews=train_data.loc[train_data.sentiment==1].review
# pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
# print("Positive words--",len(pos_words))


# train_neg_reviews=train_data.loc[train_data.sentiment==0].review
# neg_words_freq,neg_words=build_word_dict(train_neg_reviews)
# print("Negative words--",len(neg_words))

['us', 'so', 'me', 'for', 'here', 'converter', 'plug', 'i', 'the', 'unless', 'in', 'is', 'there', 'a', 'way', 'no', 'go', 'by', 'to', 'it', 'value', 'good', 'case', 'excellent', 'jawbone', 'great', '', 'problems', 'more', 'lasting', 'than', 'conversations', 'tied', 'minutesmajor', 'charger', 'mic', 'volume', 'right', 'jiggle', 'have', 'up', 'get', 'line', 'decent', 'if', 'of', 'fun', 'then', 'or', 'contacts', 'you', 'imagine', 'each', 'dozen', 'hundred', 'one', 'several', 'sending', 'them', 'this', 'are', 'must', 'razr', 'owneryou', 'needless', 'my', 'wasted', 'say', 'money', 'what', 'time', 'and', 'waste', 'sound', 'quality', 'very', 'from', 'original', 'when', 'he', 'extended', 'going', 'impressed', 'battery', 'was', 'seperated', 'notice', 'static', 'ft', 'excessive', 'were', 'two', 'mere', 'started', 'garbled', 'headset', 'though', 'at', 'odd', 'all', 'comfortable', 'clip', 'as', 'ear', 'design', 'not', 'highly', 'recommend', 'any', 'has', 'who', 'tooth', 'phone', 'blue', 'do', 'be'

{'us': 2,
 'so': 31,
 'me': 21,
 'for': 106,
 'here': 4,
 'converter': 1,
 'plug': 11,
 'i': 245,
 'the': 365,
 'unless': 2,
 'in': 80,
 'is': 189,
 'there': 11,
 'a': 169,
 'way': 7,
 'no': 25,
 'go': 6,
 'by': 7,
 'to': 146,
 'it': 213,
 'value': 5,
 'good': 71,
 'case': 26,
 'excellent': 26,
 'jawbone': 3,
 'great': 91,
 '': 102,
 'problems': 12,
 'more': 18,
 'lasting': 2,
 'than': 25,
 'conversations': 3,
 'tied': 1,
 'minutesmajor': 1,
 'charger': 14,
 'mic': 2,
 'volume': 9,
 'right': 11,
 'jiggle': 1,
 'have': 62,
 'up': 22,
 'get': 16,
 'line': 3,
 'decent': 4,
 'if': 29,
 'of': 98,
 'fun': 1,
 'then': 14,
 'or': 22,
 'contacts': 2,
 'you': 54,
 'imagine': 1,
 'each': 1,
 'dozen': 1,
 'hundred': 1,
 'one': 35,
 'several': 6,
 'sending': 4,
 'them': 12,
 'this': 186,
 'are': 33,
 'must': 4,
 'razr': 5,
 'owneryou': 1,
 'needless': 1,
 'my': 113,
 'wasted': 2,
 'say': 5,
 'money': 17,
 'what': 16,
 'time': 25,
 'and': 243,
 'waste': 13,
 'sound': 35,
 'quality': 46,
 'very': 91,

In [16]:
allwords_freq,allwords_train=build_word_dict(X_train)
for word in allwords_freq:
        print(word,"--",allwords_freq[word],"     Probability of",word,"  is =",allwords_freq[word]/len(train_data))
       

us -- 2      Probability of us   is = 0.0022222222222222222
so -- 31      Probability of so   is = 0.034444444444444444
me -- 21      Probability of me   is = 0.023333333333333334
for -- 106      Probability of for   is = 0.11777777777777777
here -- 4      Probability of here   is = 0.0044444444444444444
converter -- 1      Probability of converter   is = 0.0011111111111111111
plug -- 11      Probability of plug   is = 0.012222222222222223
i -- 245      Probability of i   is = 0.2722222222222222
the -- 365      Probability of the   is = 0.40555555555555556
unless -- 2      Probability of unless   is = 0.0022222222222222222
in -- 80      Probability of in   is = 0.08888888888888889
is -- 189      Probability of is   is = 0.21
there -- 11      Probability of there   is = 0.012222222222222223
a -- 169      Probability of a   is = 0.18777777777777777
way -- 7      Probability of way   is = 0.0077777777777777776
no -- 25      Probability of no   is = 0.027777777777777776
go -- 6      Probab

reception -- 15      Probability of reception   is = 0.016666666666666666
jabra -- 9      Probability of jabra   is = 0.01
fit -- 12      Probability of fit   is = 0.013333333333333334
piece -- 9      Probability of piece   is = 0.01
an -- 16      Probability of an   is = 0.017777777777777778
megapixels -- 1      Probability of megapixels   is = 0.0011111111111111111
relatively -- 1      Probability of relatively   is = 0.0011111111111111111
renders -- 1      Probability of renders   is = 0.0011111111111111111
rated -- 1      Probability of rated   is = 0.0011111111111111111
resolution -- 2      Probability of resolution   is = 0.0022222222222222222
impressive -- 2      Probability of impressive   is = 0.0022222222222222222
fall -- 3      Probability of fall   is = 0.0033333333333333335
high -- 4      Probability of high   is = 0.0044444444444444444
such -- 3      Probability of such   is = 0.0033333333333333335
images -- 1      Probability of images   is = 0.0011111111111111111
althou

give -- 3      Probability of give   is = 0.0033333333333333335
youd -- 1      Probability of youd   is = 0.0011111111111111111
effective -- 1      Probability of effective   is = 0.0011111111111111111
recieve -- 1      Probability of recieve   is = 0.0011111111111111111
waiting -- 2      Probability of waiting   is = 0.0022222222222222222
prompt -- 1      Probability of prompt   is = 0.0011111111111111111
especially -- 5      Probability of especially   is = 0.005555555555555556
stupid -- 1      Probability of stupid   is = 0.0011111111111111111
cradles -- 1      Probability of cradles   is = 0.0011111111111111111
keep -- 4      Probability of keep   is = 0.0044444444444444444
kits -- 1      Probability of kits   is = 0.0011111111111111111
comes -- 4      Probability of comes   is = 0.0044444444444444444
excelent -- 1      Probability of excelent   is = 0.0011111111111111111
noticed -- 2      Probability of noticed   is = 0.0022222222222222222
nicer -- 1      Probability of nicer   is

shape -- 1      Probability of shape   is = 0.0011111111111111111
cut -- 1      Probability of cut   is = 0.0011111111111111111
soft -- 1      Probability of soft   is = 0.0011111111111111111
tight -- 1      Probability of tight   is = 0.0011111111111111111
copier -- 1      Probability of copier   is = 0.0011111111111111111
sent -- 1      Probability of sent   is = 0.0011111111111111111
sizes -- 2      Probability of sizes   is = 0.0022222222222222222
anywhere -- 1      Probability of anywhere   is = 0.0011111111111111111
sold -- 1      Probability of sold   is = 0.0011111111111111111
units -- 1      Probability of units   is = 0.0011111111111111111
pros -- 1      Probability of pros   is = 0.0011111111111111111
classy -- 1      Probability of classy   is = 0.0011111111111111111
provides -- 1      Probability of provides   is = 0.0011111111111111111
krussel -- 1      Probability of krussel   is = 0.0011111111111111111
toactivate -- 1      Probability of toactivate   is = 0.001111111111

In [17]:
def fit(train_data,X_train):

    train_pos_reviews=train_data.loc[train_data.sentiment==1].review
    pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
    train_neg_reviews=train_data.loc[train_data.sentiment==0].review
    neg_words_freq,neg_words=build_word_dict(train_neg_reviews)

    neg_words_freq.pop('', None)
    neg_condi_prob={}
    for word in neg_words_freq:
        #print(word,"--",neg_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",neg_words_freq[word]/len(train_neg_reviews))
        neg_condi_prob[word]=neg_words_freq[word]/len(train_neg_reviews) 
        
    pos_words_freq.pop('', None)
    pos_condi_prob={}
    for word in pos_words_freq:
        #print(word,"--",pos_words_freq[word],"      Conditional probability of",word," given sentiment is pos=",pos_words_freq[word]/len(train_pos_reviews))
        pos_condi_prob[word]=pos_words_freq[word]/len(train_pos_reviews)
    
    
    return pos_condi_prob,neg_condi_prob
    
pos_condi_prob_train,neg_condi_prob_train=fit(train_data,X_train)   


In [18]:
#top positive words


dict1 = pos_condi_prob_train
sorted_dict = {}
sorted_keys = sorted(dict1, key=dict1.get,reverse=True)  

for w in sorted_keys:
    sorted_dict[w] = dict1[w]
 
sorted_dict.pop('i',None)
def take(n, iterable):
    
    return list(islice(iterable, n))

n_items_pos = take(10, sorted_dict.items())
print(n_items_pos)

[('the', 0.37554585152838427), ('and', 0.3231441048034934), ('is', 0.24017467248908297), ('it', 0.22270742358078602), ('this', 0.21397379912663755), ('great', 0.18995633187772926), ('a', 0.18777292576419213), ('phone', 0.1703056768558952), ('to', 0.14847161572052403), ('very', 0.14192139737991266)]


In [19]:
#top negative words


dict1 = neg_condi_prob_train
sorted_dict = {}
sorted_keys = sorted(dict1, key=dict1.get,reverse=True)  

for w in sorted_keys:
    sorted_dict[w] = dict1[w]
    
n_items_neg = take(10, sorted_dict.items())
print(n_items_neg)


[('the', 0.43665158371040724), ('i', 0.27149321266968324), ('it', 0.251131221719457), ('and', 0.2149321266968326), ('this', 0.19909502262443438), ('not', 0.19457013574660634), ('a', 0.18778280542986425), ('is', 0.17873303167420815), ('to', 0.17647058823529413), ('phone', 0.14027149321266968)]


In [20]:
def score(real, prediction):
    count = 0
    for i in range(len(real)):
        if real[i] == prediction[i]:
            count+=1
    accuracy=count/len(real)*100
    return accuracy

In [21]:
def pred(data,ans,pos_condi_prob,neg_condi_prob):
    y_preds=[]
    for sentence in data:
        sentence=text_filtering(sentence)
        pos_lik=1
        neg_lik=1
        for word in sentence.split(' '):
            word=word.lower()
            if word not in pos_condi_prob.keys():
                pos_condi_prob[word]=0
            if word not in neg_condi_prob.keys():
                neg_condi_prob[word]=0
            pos_lik*=pos_condi_prob[word]
            neg_lik*=neg_condi_prob[word]
        pos_posterior=pos_lik*prior_probs[1]
        neg_posterior=neg_lik*prior_probs[0]
        if pos_posterior>neg_posterior:
            y_preds.append(1)
        else:
            y_preds.append(0)
    
    return score(ans.tolist(),y_preds)
print("Accuracy on training data--",pred(X_train,y_train,pos_condi_prob_train,neg_condi_prob_train) )   
#print("Accuracy on development data--",pred(X_dev,y_dev,pos_condi_prob_train,neg_condi_prob_train))
print("Accuracy on test data--",pred(X_test,y_test,pos_condi_prob_train,neg_condi_prob_train))

Accuracy on training data-- 93.33333333333333
Accuracy on test data-- 74.0


In [22]:
def k_fold_acc(dataset):         # here k is 5
    
    dat_split=np.array_split(dataset,5)
    review_split=np.array_split(dataset['review'],5)
    senti_split=np.array_split(dataset['sentiment'],5)
    fold_acc=[]
    
    
    arrd1= np.concatenate((dat_split[1],dat_split[2],dat_split[3],dat_split[4]))
    arrx1=np.concatenate((review_split[1],review_split[2],review_split[3],review_split[4]))
    dfd1 = pd.DataFrame(arrd1, columns = ['review','sentiment'])
    dfx1= df = pd.DataFrame(arrx1, columns = ['review'])
    pos_condi_prob_dev,neg_condi_prob_dev=fit(dfd1,dfx1)
    acc=pred(review_split[0],senti_split[0],pos_condi_prob_dev,neg_condi_prob_dev)    
    fold_acc.append(acc)

    
    arrd2= np.concatenate((dat_split[0],dat_split[2],dat_split[3],dat_split[4]))
    arrx2=np.concatenate((review_split[0],review_split[2],review_split[3],review_split[4]))
    dfd2 = pd.DataFrame(arrd2, columns = ['review','sentiment'])
    dfx2= df = pd.DataFrame(arrx2, columns = ['review'])
    pos_condi_prob_dev,neg_condi_prob_dev=fit(dfd2,dfx2)
    acc=pred(review_split[1],senti_split[1],pos_condi_prob_dev,neg_condi_prob_dev)    
    fold_acc.append(acc)
    
    arrd3= np.concatenate((dat_split[0],dat_split[1],dat_split[3],dat_split[4]))
    arrx3=np.concatenate((review_split[0],review_split[1],review_split[3],review_split[4]))
    dfd3 = pd.DataFrame(arrd3, columns = ['review','sentiment'])
    dfx3= df = pd.DataFrame(arrx3, columns = ['review'])
    pos_condi_prob_dev,neg_condi_prob_dev=fit(dfd3,dfx3)
    acc=pred(review_split[2],senti_split[2],pos_condi_prob_dev,neg_condi_prob_dev)    
    fold_acc.append(acc)
    
    arrd4= np.concatenate((dat_split[0],dat_split[1],dat_split[2],dat_split[4]))
    arrx4=np.concatenate((review_split[0],review_split[1],review_split[2],review_split[4]))
    dfd4 = pd.DataFrame(arrd4, columns = ['review','sentiment'])
    dfx4= df = pd.DataFrame(arrx4, columns = ['review'])
    pos_condi_prob_dev,neg_condi_prob_dev=fit(dfd4,dfx4)
    acc=pred(review_split[3],senti_split[3],pos_condi_prob_dev,neg_condi_prob_dev)    
    fold_acc.append(acc)
    

    arrd5= np.concatenate((dat_split[0],dat_split[1],dat_split[2],dat_split[3]))
    arrx5=np.concatenate((review_split[0],review_split[1],review_split[2],review_split[3]))
    dfd5 = pd.DataFrame(arrd5, columns = ['review','sentiment'])
    dfx5= df = pd.DataFrame(arrx5, columns = ['review'])
    pos_condi_prob_dev,neg_condi_prob_dev=fit(dfd5,dfx5)
    acc=pred(review_split[4],senti_split[4],pos_condi_prob_dev,neg_condi_prob_dev)    
    fold_acc.append(acc)

    return fold_acc
        
        
   
    
    
l=k_fold_acc(dev_data)   
print(l)  
print("Average--", sum(l)/len(l) )


[50.0, 60.0, 50.0, 60.0, 40.0]
Average-- 52.0


In [29]:
train_pos_reviews=train_data.loc[train_data.sentiment==1].review
train_neg_reviews=train_data.loc[train_data.sentiment==0].review

In [30]:
pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
neg_words_freq,neg_words=build_word_dict(train_neg_reviews)


In [31]:
def fit_with_smoothing(train_data,X_train):
    allwords_freq,allwords_train=build_word_dict(X_train)
    train_pos_reviews=train_data.loc[train_data.sentiment==1].review
    pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
    train_neg_reviews=train_data.loc[train_data.sentiment==0].review
    neg_words_freq,neg_words=build_word_dict(train_neg_reviews)

    neg_words_freq.pop('', None)
    neg_condi_prob={}
    for word in neg_words_freq:
        #print(word,"--",neg_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",neg_words_freq[word]/len(train_neg_reviews))
        neg_condi_prob[word]=(neg_words_freq[word]+1)/(len(train_neg_reviews) +len(pos_words) )
        
    pos_words_freq.pop('', None)
    pos_condi_prob={}
    for word in pos_words_freq:
        #print(word,"--",pos_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",pos_words_freq[word]/len(train_pos_reviews))
        pos_condi_prob[word]=(pos_words_freq[word]+1)/(len(train_pos_reviews) + len(neg_words))
    
    
    return pos_condi_prob,neg_condi_prob
    
pos_condi_prob,neg_condi_prob=fit_with_smoothing(train_data,X_train)    

In [32]:
def pred(data,ans):
    y_preds=[]
    for sentence in data:
        sentence=text_filtering(sentence)
        pos_lik=1
        neg_lik=1
        
        for word in sentence.split(' '):
            word=word.lower()
            if word not in pos_condi_prob.keys():
                pos_condi_prob[word]=1/(len(train_pos_reviews)+len(pos_words))
            if word not in neg_condi_prob.keys():
                neg_condi_prob[word]=1/(len(train_neg_reviews)+len(neg_words))
            pos_lik*=pos_condi_prob[word]
            neg_lik*=neg_condi_prob[word]
        pos_posterior=pos_lik*prior_probs[1]
        neg_posterior=neg_lik*prior_probs[0]
        if pos_posterior>neg_posterior:
            y_preds.append(1)
        else:
            y_preds.append(0)
    
    return score(ans.tolist(),y_preds)
print("Accuracy on training data--",pred(X_train,y_train) )   
print("Accuracy on development data--",pred(X_dev,y_dev))
print("Accuracy on test data--",pred(X_test,y_test))

Accuracy on training data-- 95.66666666666667
Accuracy on development data-- 80.0
Accuracy on test data-- 82.0
